### Generating graph

In [17]:
import random
def generateGraph(nodes, chance_for_zero):
    net = {}
    mat = [[0 for _ in range(nodes)] for _ in range(nodes)]
    distMat = [[0 for _ in range(nodes)] for _ in range(nodes)]
    for i in range(nodes):
        for j in range(nodes):
            if i == j:
                mat[i][j] = 0
                distMat[i][j] = 0
            else:
                p = random.random()
                if p > chance_for_zero:
                    # mat[i][j] = random.randint(1, 1000)
                    mat[i][j] = 1
                    distMat[i][j] = mat[i][j]
                else:
                    mat[i][j] = 0
                    distMat[i][j] = 2000
    net['noNodes'] = nodes
    net['noEdges'] = nodes * (nodes - 1)
    net['mat'] = mat
    net['distMat'] = distMat
    net['type'] = 'generated'
    return net

### GML Read

In [18]:
#readFromGML
import networkx as nx
import numpy as np
# 
def readFromGML(filePath):
    G = nx.read_gml(filePath, label='id')
    G = nx.convert_node_labels_to_integers(G, first_label=0, ordering='default', label_attribute=None)

    adj_list = G.adj
    #print(adj_list)
    net = {}
    net['adjList'] = adj_list
    net['noNodes'] = G.number_of_nodes()
    net['noEdges'] = G.number_of_edges()
    net['nodes'] = list(G.nodes())
    net['edges'] = G.edges()
    A = nx.to_numpy_matrix(G)

    net['mat'] = A
    net['distMat'] = [[0 for _ in range(net['noNodes'])] for _ in range(net['noNodes'])]
    for i in range(net['noNodes']):
        for j in range(net['noNodes']):
            if net['mat'][i,j] == 0.0:
                net['distMat'][i][j] = 2000
            else:
                net['distMat'][i][j] = net['mat'][i,j]

    deg = dict(G.degree())
    net['degrees'] = np.array(list(deg.values()))
    
    return net

### File Read

In [19]:
import networkx as nx
import math
class Coord:
    def __init__(self, node, x, y):
        self.__node = node
        self.__x = x
        self.__y = y

    @property
    def x(self):
        return self.__x
    @property
    def y(self):
        return self.__y

def readFromFile(filePath):
    net = {}
    G = nx.Graph()
    f = open(filePath, "r")
    # !for mona_lisa_100k.tsp
    n = 100000
    # mat = [[0 for _ in range(n)] for _ in range(n)]
    net['noNodes'] = n
    net['type'] = 'file'
    nodes = []
    cnt = 0
    for line in f:
        coords = line.strip().split(" ")
        i = int(coords[0])
        x = int(coords[1])
        y = int(coords[2])
        coord = Coord(i, x, y)
        nodes.append(coord)
    net['nodes'] = nodes
    return net

### Plotting

In [20]:
#plotNetwork
import matplotlib.pyplot as plt 
import matplotlib 
import warnings 
import networkx as nx
import numpy as np

# plot a network 
def plotNetwork(network, path):
    np.random.seed(123) #to freeze the graph's view (networks uses a random view)
    if network['type'] != 'file':
        A=np.matrix(network['mat'])
        G=nx.from_numpy_matrix(A)
        node_colors = np.zeros(G.number_of_nodes())
        i = 0
        while i < len(path):
            node_colors[path[i]] = 1
            i += 1
    
    edges = set()
    i = 1
    edges.add((network['source'], path[0]))
    while i < len(path):
        edges.add((path[i - 1], path[i]))
        i += 1
    edges.add((path[i - 1], network['source']))
    
    if network['type'] != 'file':
        edge_colors = np.zeros(G.number_of_edges())
        cnt = 0
        for i, edge in enumerate(G.edges()):
            if edge in edges or (edge[1], edge[0]) in edges:
                cnt += 1
                edge_colors[i] = 1
        percent = cnt / network['noNodes']
        print(f'% of edges: {percent}')
        colors = ['red' if val else 'black' for val in edge_colors]

    if network['noNodes'] <= 500:
        pos = nx.spring_layout(G)  # compute graph layout
        plt.figure(figsize=(10, 10))  # image is 8 x 8 inches 
        labels = {i: i for i in G.nodes()}
        nx.draw_networkx_nodes(G, pos, node_size=80, cmap=plt.cm.RdYlBu)
        nx.draw_networkx_edges(G, pos, alpha=0.5, edge_color=colors)
        nx.draw_networkx_labels(G, pos)

        plt.show(G)

### Fitness function
Sum of all weights on the path, where a path entry is represented by {u, v, w}, meaning there is an edge between u and v of weight w

In [21]:
def calc_euclidian_distance(coord1, coord2):
    return math.sqrt((coord2.x - coord1.x) ** 2 + (coord2.y - coord1.y) ** 2)

def sumWeights(path, network):

    source = network['source']
    V = network['noNodes'] - 1
    if network['type'] != 'file':
        adjMatrix = network['distMat']
        sum = adjMatrix[source][path[0]]
        for i in range(1, V - 1):
            sum += adjMatrix[path[i]][path[i + 1]]
        sum += adjMatrix[path[V - 1]][source]
        #print(sum)
        return sum
    else:
        nodes = network['nodes']
        sum = calc_euclidian_distance(nodes[source], nodes[path[0]])
        for i in range(1, V - 1):
            sum += calc_euclidian_distance(nodes[path[i]], nodes[path[i + 1]])
        sum += calc_euclidian_distance(nodes[path[V - 1]], nodes[source])
        return sum
    # return 1 / sum

### Chromosome definition
<strong>Initialization</strong> - Permutation of all nodes except the source node<br/>
<strong>Selection</strong> - First best 2 individuals <br/>
<strong>Crossover</strong> - Order 1 crossover (OX1) / Partially-mapped crossover<br/>
<strong>Mutation</strong> - Swap mutation<br/>
<strong>Replacement</strong> - If more fit, will replace the worst in that generation<br/>
<strong>Termination condition</strong> - Number of generations<br/>
<a href="https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35">credit

In [22]:
import random
import numpy as np

class Chromosome:
    def __init__(self, problParam):
        self.__problParam = problParam
        self.__fitness = 0.0
        #initiating a ndarray of dimension 1 with -1
        self.__repres = np.full((self.__problParam['network']['noNodes'] - 1, ), -1)


    @property
    def fitness(self):
        return self.__fitness
    
    @property
    def repres(self):
        return self.__repres
    
    @fitness.setter
    def fitness(self, fit = 0.0):
        self.__fitness = fit
    
    @repres.setter
    def repres(self, l = []):
        self.__repres = l
        
    def initPath(self):
        V = self.__problParam['network']['noNodes']
        # *array with values (0, 1, .. V - 1)
        arr = np.arange(V)

        # *shuffling the array
        shuffled_arr = np.random.permutation(arr)

        # !filtering out using boolean indexing
        source = self.__problParam['source']
        self.__repres = shuffled_arr[shuffled_arr != source]
    
    def mutation(self):
        # *selecting 2 random indices 
        i, j = np.random.choice(self.__repres.size, size=2, replace=False)
        # *swapping their values
        self.__repres[i], self.__repres[j] = self.__repres[j], self.__repres[i]

    def order_1_crossover(self, c):
        V = self.__problParam['network']['noNodes'] - 1
        segment_index = random.randint(0, V // 2)
        segment_size = random.randint(1, V // 2)

        genes_segment_p1 = self.__repres[segment_index : segment_index + segment_size]
        genes_segment_p2 = c.repres[segment_index : segment_index + segment_size]

        offspring1 = Chromosome(self.__problParam)
        offspring2 = Chromosome(self.__problParam)

        offspring1.repres[segment_index : segment_index + segment_size] = genes_segment_p1
        offspring2.repres[segment_index : segment_index + segment_size] = genes_segment_p2

        negative_indices = np.where(offspring1.repres == -1)[0]
        negative_index = negative_indices[0]
        while negative_index < V:
            if negative_index == segment_index:
                negative_index += segment_size
            else:
                for i in range(V):
                    if negative_index == segment_index:
                        break
                    if c.repres[i] not in genes_segment_p1 and c.repres[i] not in offspring1.repres:
                        offspring1.repres[negative_index] = c.repres[i]
                        negative_index += 1

        negative_index = negative_indices[0]
        while negative_index < V:
            if negative_index == segment_index:
                negative_index += segment_size
            else:
                for i in range(V):
                    if negative_index == segment_index:
                        break
                    if self.__repres[i] not in genes_segment_p2 and self.__repres[i] not in offspring2.repres:
                        offspring2.repres[negative_index] = self.__repres[i]
                        negative_index += 1

        return offspring1, offspring2
    


    def partially_mapped_crossover(self, c):
        # *substracting 1 because of the source node outside of our permutations array
        V = self.__problParam['network']['noNodes'] - 1
        index = random.randint(0, V // 2)

        genes_from_parent1 = self.__repres[index : index + V // 2]
        genes_from_parent2 = c.repres[index : index + V // 2]

        # *initiating the offspring
        offspring1 = Chromosome(self.__problParam)
        offspring2 = Chromosome(self.__problParam)

        # *adding to their repre
        offspring1.repres[index : index + V // 2] = c.repres[index : index + V // 2]
        offspring2.repres[index : index + V // 2] = self.__repres[index : index + V // 2]

        # *array of indices there's -1 in the representation of the offsprings
        # *initially they will have -1's in the same places => we can choose arbitrarily which representation to evaluate
        negative_indices_in_repres = np.where(offspring1.repres == -1)[0]

        # ! applying the partially-mapped crossover algorithm for filling up the empty spaces 

        # *for offspring1
        for index in negative_indices_in_repres:
            gene_parent1 = self.__repres[index]
            while gene_parent1 in genes_from_parent2:
                index_gene_in_parent2 = np.where(genes_from_parent2 == gene_parent1)[0][0]
                gene_parent1 = genes_from_parent1[index_gene_in_parent2]
            offspring1.repres[index] = gene_parent1

        # *for offspring2
        for index in negative_indices_in_repres:
            gene_parent2 = c.repres[index]
            while gene_parent2 in genes_from_parent1:
                index_gene_in_parent1 = np.where(genes_from_parent1 == gene_parent2)[0][0]
                gene_parent2 = genes_from_parent2[index_gene_in_parent1]
            offspring2.repres[index] = gene_parent2
        
        return offspring1, offspring2

    def ordered_crossover(self, c):
        V = self.__problParam['network']['noNodes']
        # *filling offspring with -1
        offspring = Chromosome(self.__problParam)
        offspring.repres = np.full_like(self.__repres, -1)

        # *choosing random noSgements to copy from .self
        # *up to  (v - 1) / 2
        noSegments = np.random.randint(1, V // 4)
        # *segment sizes
        k = 3
        # *choosing random V - 1 indices, then sorting them for convenience
        segment_indices = np.sort(np.random.choice(V - 1, size=noSegments, replace=False))
        # *list of tuples, where tuples are {segments, start_segment_index} to remember the position for offspring copy
        segments = [(self.__repres[segment_indices[i]:segment_indices[i] + k], segment_indices[i]) for i in range(noSegments - 1)]

        # *sorting by indices
        segments = sorted(segments, key = lambda x: x[1])

        # *copying the segments into the offspring
        for segment in segments:
            start = segment[1]
            end = start + len(segment[0])
            offspring.repres[start:end] = segment[0]

        # *filling in remaining slots from other chromosome: c
        index = 0
        while offspring.repres[index] != -1:
            index += 1
        for gene in c.repres:
            if gene not in self.__repres:
                offspring.repres[index] = gene
                index += 1

        return offspring        

    def evaluate(self):
        self.fitness = self.__problParam['function'](self.__repres, self.__problParam['network'])

### Genetic algorithm

In [23]:
import heapq
import ipywidgets as widgets

class GA:
    def __init__(self, problParam = None):
        self.__problParam = problParam
        self.__population = []

    @property
    def population(self):
        return self.__population

    def inititialization(self):
        for chrCount in range(self.__problParam['popSize']):
            c = Chromosome(self.__problParam)
            c.initPath()
            self.__population.append(c)

    def evaluation(self):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres, self.__problParam['network'])

    def nextGeneration(self):
        fitness = lambda x: x.fitness
        #select the best 2 individuals
        entries = np.random.choice(len(self.__population), size=2, replace=False)
        p1 = self.__population[entries[0]]
        p2 = self.__population[entries[1]]
        # *generating offspring through crossover and mutation
        offspring1, offspring2 = p1.partially_mapped_crossover(p2)
        offspring1.mutation()
        offspring2.mutation()
        # *evaluating the fitness of the new offspring
        offspring1.evaluate()
        offspring2.evaluate()
        
        # *getting the least fit individual
        # *sorting ascending > getting lowest fitness
        # ?worst_indices = sorted(range(len(self.__population)), key=lambda i:self.__population[i].fitness, reverse=True)[:1]
        worst_indices = sorted(range(len(self.__population)), key=lambda i:self.__population[i].fitness, reverse=True)[:1]
        chosen_offspring = offspring1
        if offspring2.fitness < offspring1.fitness:
            chosen_offspring = offspring2

        if chosen_offspring.fitness < self.__population[worst_indices[0]].fitness:
            self.__population[worst_indices[0]] = chosen_offspring
        
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            # ?if c.fitness > best.fitness:
            if c.fitness < best.fitness:
                best = c
        best_list = []
        for c in self.__population:
            if c.fitness == best.fitness:
                best_list.append(c)
        return best_list


### Running the algorithm

In [24]:
#runningAlgorithm

import ipywidgets as widgets
import time
def applyGA(network, source, destination, iterations):
    network['source'] = source
    problParam = {'popSize': 50, 'network': network, 'source': source, 'destination': destination, 'function': sumWeights}
    ga = GA(problParam)
    ga.inititialization()
    ga.evaluation()

    progressBar = widgets.IntProgress(min=1, max=iterations)
    display(progressBar)
    for _ in range(iterations):
        ga.nextGeneration()
        progressBar.value += 1
        time.sleep(0.01)

    best_list = ga.bestChromosome()
    unique_paths = []
    seen_paths = set()
    for path in best_list:
        path_repr = tuple(path.repres)
        if path_repr not in seen_paths:
            unique_paths.append(path)
            seen_paths.add(path_repr)
        
    if network['type'] != 'file': 
        for path in unique_paths:
            print(f'fitness: {path.fitness}, repres: {path.repres}')
            plotNetwork(network, path=path.repres)
    else:
        print(f'fitness: {path.fitness}, repres: {path.repres}')
        f = open("mona-lisa-out.txt", "a")
        for path in unique_paths:
            for node in path.repres:
                f.write(str(node) + " ")

# network = readFromGML('data\\real\\graph.gml')
# network = readFromFile("data\\tsp_dataset\\five_d.txt")
# print('generating graph')
# network = generateGraph(nodes=5, chance_for_zero=0.5)
# print('generated graph')
# applyGA(network, source = 4, destination =4, iterations=30)
# print('generating graph')
# network = generateGraph(nodes=20, chance_for_zero=0.5)
# print('generated graph')
# applyGA(network, source = 4, destination =4, iterations=30)
# print('generating graph')
# network = generateGraph(nodes=600, chance_for_zero=0.5)
# print('generated graph')
# applyGA(network, source = 4, destination =4, iterations=30)
network = readFromFile('data\\tsp_dataset\\mona-lisa100K.tsp')
applyGA(network, source = 4, destination =4, iterations=10)



IntProgress(value=1, max=10, min=1)

fitness: 988313861.7268904, repres: [94727 47579 93148 ... 46866 24439 76954]
